## 0 - Imports

In [2]:
import scipy.io as spio
import pandas as pd
import numpy as np
from numpy import absolute as nabs

## 1 - Load, clean, test data

In [37]:
def impmat(fname = 'M_processed.mat', writ = True):
    ''' import matlab crap, and turn it to pickles (or return panda df)'''
    mat = spio.loadmat(fname, squeeze_me=True)
    M = mat['M'] 
    head = ['time','ndc1','ndc2','ndc3','Trade_Partner_Name',
    'Distribution_Center_State','NDC','Distribution_Center_ID_(IC)',
    'Distribution_Center_Zip','Eff_Inv_(EU)','Eff_Inv_(PU)',
    'Qty_Ord_(EU)','Qty_Ord_(PU)']
    # get rid of ndc 1,2,3 because they're pieces of NCD
    # also get rid of purchase units, just use eatable
    head_adj = [head[0]] + head[4:10] + [head[-2]]
    data = pd.DataFrame(M, columns=head)
    data["time"] = pd.to_datetime(data["time"], format='%Y%m%d', errors='coerce')

    if writ: # h5 allows your variable to be external
        dt = pd.HDFStore("drugdata.h5") # don't need to import/export! warning, though: huge
        dt['dat'] = data[head_adj] #
    return(data)


In [38]:
def test_hd5(p = 0, q = 0):
    """test data and run answers to intro quiz
    p is to print head of dataframe
    q prints quiz answers
    doesn't return anything
    mostly for access examples"""
    dt = pd.HDFStore("drugdata.h5")["dat"]

    header = dt.columns.tolist()
    # thanks @brock
    def q1(df):
        return(df.Trade_Partner_Name.unique())
    
    def q2(df):
        q2 = df.groupby('Trade_Partner_Name')['Distribution_Center_ID_(IC)'].nunique()
        q2max = q2.max()
        return(q2[q2 == q2max])
    
    def q3(df):
        q3df = df.loc[df["time"].dt.year == 2011] # can also use dt.month
        q3TotalSales = q3df.groupby('NDC')['Qty_Ord_(PU)'].sum()
        #print(q3TotalSales)
        q3sorted = q3TotalSales.sort_values(ascending = False).head()
        return(q3sorted)
    
    def q4(df):
        q4 = df['NDC'].value_counts()
        NDCLessThan60 = q4[q4 < 60]
        if (NDCLessThan60.size == 0):
            return(None)
        else:
            return(NDCLessThan60.size)
        
    def q5(df):
        q5 =  df.groupby('NDC')['Qty_Ord_(PU)'].std()
        q5max = q5.max()
        NDCHighestVariance = q5[q5 == q5max]
        return(NDCHighestVariance)
    
    def q6(df):
        q6 = df.groupby('NDC')['Qty_Ord_(PU)'].nunique()
        q6ZeroDemand = q6[q6 == 0]
        if (q6ZeroDemand.size == 0):
            return(None)
        else:
            return(q6ZeroDemand.size)
    
    if p:
        for col in header:
            print(dt[col].head())
    if q:
        answers = [q1(dt), q2(dt), q3(dt), q4(dt), q5(dt), q6(dt)]
        for i, ans in enumerate(answers):
            try:
                print('Question %d'%(i+1),  ans)
            except:
                print('Question %d'%(i+1) + str(ans))

In [39]:
#impmat() # uncomment if never built h5 file
test_hd5() # add p=1 or q = 1 to print stuff

In [40]:
def rem_neg_vals():
    ''' if you've just imported from the mat file,
    you need to run this to change the neg vals to 0 '''
    df = pd.HDFStore("drugdata.h5")["dat"]
    # set negative values to 0
    df.loc[df['Eff_Inv_(EU)'] < 0,'Eff_Inv_(EU)'] = 0
    df.loc[df['Qty_Ord_(EU)'] < 0,'Qty_Ord_(EU)'] = 0
    return(True)

In [41]:
rem_neg_vals()

True

## 2 - Early queries

In [61]:
def how_many_sales(thr):
    ''' in: minimum contributing percentage threshold
        out: ID, % of drugs above thr'''
    df = pd.HDFStore("drugdata.h5")["dat"]
    ind_total = df.groupby('NDC')['Qty_Ord_(EU)'].sum()
    sortsales = ind_total.sort_values(ascending = False)
    #print(sortsales)
    total = sum(ind_total.values)
    perc_total = 100 * sortsales / total
    clipped_above_total = perc_total[perc_total > thr]
    print(len(clipped_above_total), sum(clipped_above_total.values))
    return(clipped_above_total)

In [62]:
how_many_sales(2)

11 78.4506510137


NDC
55.0     22.399959
4.0      16.361364
154.0     9.998171
109.0     6.040427
25.0      5.915590
125.0     3.942696
7.0       3.572012
85.0      3.007674
141.0     2.970900
62.0      2.224088
157.0     2.017771
Name: Qty_Ord_(EU), dtype: float64

In [117]:
def weeks():
    df = pd.HDFStore("drugdata.h5")["dat"]
    ## 2008, at least, does indeed have 52 weeks
    ## '07 has 27, '17 has 34
    for i in range(2007, 2018):
        y2k = df.loc[df.time.dt.year == (i)]
        wy2k = y2k.groupby("time").nunique()
        ly = len(wy2k["time"])
        print(i, ly)
        if ly != 52:
            print(y2k.time.unique())
    #print(df.time.nunique()) # 530 unique dates
    #print(df.time.unique())
    

In [118]:
weeks()

2007 27
['2007-07-01T00:00:00.000000000' '2007-07-08T00:00:00.000000000'
 '2007-07-15T00:00:00.000000000' '2007-07-22T00:00:00.000000000'
 '2007-07-29T00:00:00.000000000' '2007-08-05T00:00:00.000000000'
 '2007-08-12T00:00:00.000000000' '2007-08-19T00:00:00.000000000'
 '2007-08-26T00:00:00.000000000' '2007-09-02T00:00:00.000000000'
 '2007-09-09T00:00:00.000000000' '2007-09-16T00:00:00.000000000'
 '2007-09-23T00:00:00.000000000' '2007-09-30T00:00:00.000000000'
 '2007-10-21T00:00:00.000000000' '2007-10-28T00:00:00.000000000'
 '2007-11-11T00:00:00.000000000' '2007-11-18T00:00:00.000000000'
 '2007-11-25T00:00:00.000000000' '2007-12-02T00:00:00.000000000'
 '2007-12-09T00:00:00.000000000' '2007-12-16T00:00:00.000000000'
 '2007-12-23T00:00:00.000000000' '2007-12-30T00:00:00.000000000'
 '2007-10-07T00:00:00.000000000' '2007-10-14T00:00:00.000000000'
 '2007-11-04T00:00:00.000000000']
2008 52
2009 52
2010 52
2011 52
2012 53
['2012-09-16T00:00:00.000000000' '2012-09-09T00:00:00.000000000'
 '2012-0

## 3 - The meaty bits!

In [25]:
def sales(df, t, co):
    """in: df=data object, t=time of interest, co=partner, 
    want to return the list of sales per location"""
    q3df = df.loc[df["time"].dt.week = t] # can also use dt.month
    
    
    

Question 1: [  4.  20.   2.   3.  21.  15.  12.   9.   5.  14.   0.  22.  26.  27.   8.
  24.  10.  13.  11.  16.   7.  19.  23.  17.  25.   6.   1.  18.]


In [42]:
def smape(f, d):
    ''' symmetric mean absolute percentage error
    in: vectors f = y_hat, d = y 
    out: the smape, yo '''
    n = len(f)
    num = np.sum(nabs(f - d))
    denom = np.sum(nabs(f) + nabs(d))
    return((1/n) * num/denom)

## Plots

In [ ]:
def main():
    dt = pd.HDFStore("drugdata.h5")["dat"]
    start_date = pd.to_datetime("20110101")
    